In [64]:
from PIL import Image
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [65]:
def square_for_image(path: str):
    # Đọc hình ảnh gốc
    image_path = path  # Thay bằng đường dẫn đến hình ảnh của bạn
    original_image = Image.open(image_path)

    # Kích thước ảnh gốc và ảnh mới
    original_size = original_image.size

    # Kích thước ảnh mới sẽ là kích thước lớn nhất của ảnh gốc
    max_width = max(original_size)

    new_size = (max_width, max_width)

    # Tạo một ảnh mới với nền đen
    new_image = Image.new("RGB", new_size, (0, 0, 0))

    # Dán ảnh gốc vào ảnh mới với khoảng trống màu đen
    new_image.paste(original_image, 
                    ((max_width - original_size[0]) // 2, (max_width - original_size[1]) // 2))

    # convert new_image to cv2 format
    new_image_cv2 = cv2.cvtColor(np.array(new_image), cv2.COLOR_RGB2BGR)

    return new_image_cv2


In [66]:
def draw_landmarks(mp_drawing, mp_pose, image : cv2, pose_landmarks : np.array):
    """
    Vẽ landmarks lên ảnh
    """
    mp_drawing.draw_landmarks(
        image,
        pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing.DrawingSpec(
            color=(255, 0, 0),  # Màu sắc của các landmark
            thickness=5,  # Độ dày của các đường nối landmark
            circle_radius=5,  # Bán kính của các điểm landmark
        ),
        connection_drawing_spec=mp_drawing.DrawingSpec(
            color=(0, 255, 0),  # Màu sắc của các đường nối
            thickness=5,  # Độ dày của các đường nối
        ),
    )

In [67]:
def extract_key_points_from_image(
    image_path: str, label: str, output_file: str, is_draw_landmarks=False
):
    # Khởi tạo MediaPipe Pose
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(
        static_image_mode=True, model_complexity=1, smooth_landmarks=True
    )

    image = square_for_image(image_path)

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Phát hiện pose
    results = pose.process(image_rgb)

    if results.pose_landmarks:
        if is_draw_landmarks:
            draw_landmarks(mp_drawing, mp_pose, image, results.pose_landmarks)
            last_dot_index = image_path.rfind(".")
            image_path_output = image_path[:last_dot_index] + "_landmarks" + image_path[last_dot_index:]
            cv2.imwrite(image_path_output, image) # Lưu ảnh đã vẽ landmark

        # Lưu lại toạ độ các key points sau khi dịch chuyển vào csv
        columns_name = ["label"]
        columns_value = [label]
        for id, landmark in enumerate(results.pose_landmarks.landmark):
            land_mark_name = mp_pose.PoseLandmark(id).name.lower()
            
            columns_name += [
                f"{ land_mark_name }_x",
                f"{ land_mark_name }_y",
                f"{ land_mark_name }_z",
            ]

            # landmark.x, landmark.y là các giá trị trước khi dịch chuyển gốc toạ độ vào giữa bức hình
            # Do đó khi đưa gốc toạ độ về giữa bức hình thì phải trừ chúng cho 0.5
            columns_value += [
                landmark.x - 0.5,
                landmark.y - 0.5,
                landmark.z,
            ]

        df_key_points = pd.DataFrame([columns_value], columns=columns_name)

        left_hip = (df_key_points["left_hip_x"], df_key_points["left_hip_y"])
        right_hip = (df_key_points["right_hip_x"], df_key_points["right_hip_y"])

        center_hip = ((left_hip[0] + right_hip[0]) / 2, (left_hip[1] + right_hip[1]) / 2)

        nose = (df_key_points["nose_x"], df_key_points["nose_y"])


        # distance between left hip and right hip
        distance = np.sqrt((center_hip[0] - nose[0])**2 + (center_hip[1] - nose[1])**2)

        scale_value = 0.5 / distance

        # Scale all key points
        for id, landmark in enumerate(results.pose_landmarks.landmark):
            land_mark_name = mp_pose.PoseLandmark(id).name.lower()
            df_key_points[f"{ land_mark_name }_x"] = df_key_points[f"{ land_mark_name }_x"] * scale_value
            df_key_points[f"{ land_mark_name }_y"] = df_key_points[f"{ land_mark_name }_y"] * scale_value

        with open(output_file, "a") as f:
            # header=f.tell()==0 sẽ ghi tiêu đề nếu file trống
            pass
            #df_key_points.to_csv(f, header=f.tell() == 0, index=False)

        return df_key_points
        
    else:
        print(f"Cannot detect pose in image {image_path}")

    pose.close()

In [68]:
df = extract_key_points_from_image(image_path=
                                "./data/1_lie_on/26.jpg",
                                label="1_lie_on",
                                output_file="key_points.csv",
                                is_draw_landmarks=True)

d:\d7thSemesterUniversity\PBL 06\pbl6\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [69]:
df.head()

,label,nose_x,nose_y,nose_z,left_eye_inner_x,left_eye_inner_y,left_eye_inner_z,left_eye_x,left_eye_y,left_eye_z,...,left_heel_z,right_heel_x,right_heel_y,right_heel_z,left_foot_index_x,left_foot_index_y,left_foot_index_z,right_foot_index_x,right_foot_index_y,right_foot_index_z
0,1_lie_on,-0.004741,-0.339577,-0.402923,-0.005197,-0.371458,-0.354782,0.003454,-0.372347,-0.35465,...,0.220519,-0.098988,0.394426,0.115957,0.191265,0.498521,0.101621,-0.172976,0.490614,-0.010314
